In [1]:
#データを時系列順に並べ、過去のデータで未来を予測する形にする
##そのために、まずは日付データを取得する
import requests
from bs4 import BeautifulSoup

race_id = '202106030811'
url = 'https://db.netkeiba.com/race/' +race_id
html = requests.get(url)
html.encoding = 'EUC-JP'
soup = BeautifulSoup(html.text, 'html.parser')

In [10]:
text = soup.find('div',attrs=['class','data_intro']).find_all('p')[0].text
text

'\n\n芝右2000m\xa0/\xa0天候 : 晴\xa0/\xa0芝 : 稍重\xa0/\xa0発走 : 15:40\n\n過去の皐月賞\n\n'

In [15]:
import re

re.findall(r'\w+', text)

['芝右2000m', '天候', '晴', '芝', '稍重', '発走', '15', '40', '過去の皐月賞']

In [16]:
text2 = soup.find('div',attrs=['class','data_intro']).find_all('p')[1].text
re.findall(r'\w+', text2)

['2021年4月18日', '3回中山8日目', '3歳オープン', '国際', '牡', '牝', '指', '定量']

In [17]:
import pandas as pd
result = pd.read_pickle('result.pickle')

In [19]:
result.index.unique()

Index(['202001010101', '202001010102', '202001010103', '202001010104',
       '202001010105', '202001010106', '202001010107', '202001010108',
       '202001010109', '202001010110',
       ...
       '202010020803', '202010020804', '202010020805', '202010020806',
       '202010020807', '202010020808', '202010020809', '202010020810',
       '202010020811', '202010020812'],
      dtype='object', length=3120)

In [20]:
race_id_list = result.index.unique()[:50]

In [36]:
import time
from tqdm.notebook import tqdm as tqdm

race_infos = {}
for race_id in tqdm(race_id_list):
    url = 'https://db.netkeiba.com/race/' +race_id
    html = requests.get(url)
    html.encoding = 'EUC-JP'
    soup = BeautifulSoup(html.text, 'html.parser')
    
    texts = soup.find('div',attrs=['class','data_intro']).find_all('p')[0].text + \
        soup.find('div',attrs=['class','data_intro']).find_all('p')[1].text
    info = re.findall(r'\w+', texts)
    #infoに含まれるtextの内容によって項目を判断
    info_dict = {}
    for text in info:
        if text in ['芝','ダート']:
            info_dict['race_type'] = text
        if '障' in text:
            info_dict['race_type'] = '障害'
        if 'm' in text:
            info_dict['course_len'] = re.findall(r'\d+',text)[0]
        if text in ['良','稍重','重','不良']:
            info_dict['ground_state'] = text
        if text in ['曇','晴','雨','小雨','小雪','雪']:
            info_dict['weather'] = text
        if '年' in text:
            info_dict['date'] = text
    race_infos[race_id] = info_dict
    time.sleep(0.5)

In [39]:
pd.DataFrame(race_infos).T

,course_len,weather,race_type,ground_state,date
202001010101,1800,曇,芝,良,2020年7月25日
202001010102,1700,曇,ダート,良,2020年7月25日
202001010103,1500,曇,芝,良,2020年7月25日
202001010104,1700,曇,ダート,良,2020年7月25日
202001010105,1200,晴,芝,良,2020年7月25日
202001010106,2000,晴,芝,良,2020年7月25日
202001010107,1000,晴,ダート,良,2020年7月25日
202001010108,1700,晴,ダート,良,2020年7月25日
202001010109,2000,晴,芝,良,2020年7月25日
202001010110,1700,晴,ダート,良,2020年7月25日


In [41]:
def scrape_race_info(race_id_list):
    race_infos = {}
    
    for race_id in tqdm(race_id_list):
        try:
            url = 'https://db.netkeiba.com/race/' +race_id
            html = requests.get(url)
            html.encoding = 'EUC-JP'
            soup = BeautifulSoup(html.text, 'html.parser')

            texts = soup.find('div',attrs=['class','data_intro']).find_all('p')[0].text + \
                soup.find('div',attrs=['class','data_intro']).find_all('p')[1].text
            info = re.findall(r'\w+', texts)
            #infoに含まれるtextの内容によって項目を判断
            info_dict = {}
            for text in info:
                if text in ['芝','ダート']:
                    info_dict['race_type'] = text
                if '障' in text:
                    info_dict['race_type'] = '障害'
                if 'm' in text:
                    info_dict['course_len'] = re.findall(r'\d+',text)[0]
                if text in ['良','稍重','重','不良']:
                    info_dict['ground_state'] = text
                if text in ['曇','晴','雨','小雨','小雪','雪']:
                    info_dict['weather'] = text
                if '年' in text:
                    info_dict['date'] = text
            race_infos[race_id] = info_dict
            time.sleep(0.5)
        except IndexError:
            continue
        except:
            break
    return race_infos

In [45]:
race_id_list = result.index.unique()
race_infos = scrape_race_info(race_id_list)

In [44]:
race_infos

{'202001010101': {'course_len': '1800',
  'weather': '曇',
  'race_type': '芝',
  'ground_state': '良',
  'date': '2020年7月25日'},
 '202001010102': {'course_len': '1700',
  'weather': '曇',
  'race_type': 'ダート',
  'ground_state': '良',
  'date': '2020年7月25日'},
 '202001010103': {'course_len': '1500',
  'weather': '曇',
  'race_type': '芝',
  'ground_state': '良',
  'date': '2020年7月25日'},
 '202001010104': {'course_len': '1700',
  'weather': '曇',
  'race_type': 'ダート',
  'ground_state': '良',
  'date': '2020年7月25日'},
 '202001010105': {'course_len': '1200',
  'weather': '晴',
  'race_type': '芝',
  'ground_state': '良',
  'date': '2020年7月25日'},
 '202001010106': {'course_len': '2000',
  'weather': '晴',
  'race_type': '芝',
  'ground_state': '良',
  'date': '2020年7月25日'},
 '202001010107': {'course_len': '1000',
  'weather': '晴',
  'race_type': 'ダート',
  'ground_state': '良',
  'date': '2020年7月25日'},
 '202001010108': {'course_len': '1700',
  'weather': '晴',
  'race_type': 'ダート',
  'ground_state': '良',
  'date':

In [51]:
race_infos = pd.DataFrame(race_infos).T

In [48]:
result

,着順,枠番,馬番,馬名,斤量,騎手,単勝,人気,性,齢,体重,体重増減
202001010101,1,6,6,ウインルーア,54.0,横山武史,16.0,3.0,牝,2,438,4
202001010101,2,2,2,アークライト,54.0,ルメール,1.9,2.0,牡,2,510,0
202001010101,3,3,3,ギャラントウォリア,54.0,池添謙一,1.8,1.0,牡,2,482,-6
202001010101,4,1,1,ジュンブーケ,52.0,亀田温心,22.2,4.0,牝,2,442,0
202001010101,5,4,4,キタノマンゲツ,54.0,藤岡康太,55.7,5.0,牡,2,426,-8
...,...,...,...,...,...,...,...,...,...,...,...,...
202010020812,11,1,1,キスラー,57.0,武豊,6.5,4.0,牡,4,476,-2
202010020812,12,8,14,ポンペイワーム,57.0,浜中俊,19.8,8.0,牡,4,522,-8
202010020812,13,8,15,アルマトップエンド,57.0,国分恭介,75.0,14.0,牡,5,510,0
202010020812,14,2,3,ノイーヴァ,55.0,太宰啓介,46.1,10.0,牝,4,488,-12


In [158]:
result_addinfo = result.merge(race_infos, left_index=True, right_index=True, how='inner')

In [159]:
result_addinfo

,着順,枠番,馬番,馬名,斤量,騎手,単勝,人気,性,齢,体重,体重増減,course_len,weather,race_type,ground_state,date
202001010101,1,6,6,ウインルーア,54.0,横山武史,16.0,3.0,牝,2,438,4,1800,曇,芝,良,2020年7月25日
202001010101,2,2,2,アークライト,54.0,ルメール,1.9,2.0,牡,2,510,0,1800,曇,芝,良,2020年7月25日
202001010101,3,3,3,ギャラントウォリア,54.0,池添謙一,1.8,1.0,牡,2,482,-6,1800,曇,芝,良,2020年7月25日
202001010101,4,1,1,ジュンブーケ,52.0,亀田温心,22.2,4.0,牝,2,442,0,1800,曇,芝,良,2020年7月25日
202001010101,5,4,4,キタノマンゲツ,54.0,藤岡康太,55.7,5.0,牡,2,426,-8,1800,曇,芝,良,2020年7月25日
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202010020812,11,1,1,キスラー,57.0,武豊,6.5,4.0,牡,4,476,-2,1700,雨,ダート,重,2020年9月6日
202010020812,12,8,14,ポンペイワーム,57.0,浜中俊,19.8,8.0,牡,4,522,-8,1700,雨,ダート,重,2020年9月6日
202010020812,13,8,15,アルマトップエンド,57.0,国分恭介,75.0,14.0,牡,5,510,0,1700,雨,ダート,重,2020年9月6日
202010020812,14,2,3,ノイーヴァ,55.0,太宰啓介,46.1,10.0,牝,4,488,-12,1700,雨,ダート,重,2020年9月6日


In [136]:
import datetime

def preprocessing(kekka):
    result = kekka.copy()
    #着順の0-9以外の文字を含むデータを除く
    result = result[-result["着順"].astype(str).str.contains("\D")] 
    result["着順"] = result["着順"].astype(int)
    #性齢を性と齢に分ける
    result["性"] = result["性齢"].map(lambda x: str(x)[0])
    result["齢"] = result["性齢"].map(lambda x: str(x)[1:]).astype(int)
    #馬体重を体重と体重増減に分ける
    result["体重"] = result["馬体重"].str.split('(',expand=True)[0].astype(int)
    result["体重増減"] = result["馬体重"].str.split('(',expand=True)[1].str[:-1].astype(int)
    
    result["単勝"] = result["単勝"].astype(float)
    result.drop(['性齢','馬体重','タイム','着差','調教師'],axis=1,inplace=True)
    
    result['date'] = pd.to_datetime(result['date'], format='%Y年%m月%d日')
    return result

In [161]:
result_addinfo['date'] = pd.to_datetime(result_addinfo['date'], format='%Y年%m月%d日')
result_addinfo['course_len'] = result_addinfo['course_len'].astype(int)

In [162]:
result_addinfo.info()

<class 'pandas.core.frame.DataFrame'>
Index: 43239 entries, 202001010101 to 202010020812
Data columns (total 17 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   着順            43239 non-null  int32         
 1   枠番            43239 non-null  int64         
 2   馬番            43239 non-null  int64         
 3   馬名            43239 non-null  object        
 4   斤量            43239 non-null  float64       
 5   騎手            43239 non-null  object        
 6   単勝            43239 non-null  float64       
 7   人気            43239 non-null  float64       
 8   性             43239 non-null  object        
 9   齢             43239 non-null  int32         
 10  体重            43239 non-null  int32         
 11  体重増減          43239 non-null  int32         
 12  course_len    43239 non-null  int32         
 13  weather       43239 non-null  object        
 14  race_type     43239 non-null  object        
 15  ground_state  43239 non

In [163]:
sorted_id_list = result_addinfo.sort_values('date').index.unique()

In [164]:
round(len(sorted_id_list)*0.7)

2184

In [165]:
train_id_list = sorted_id_list[:round(len(sorted_id_list)*0.7)]
test_id_list = sorted_id_list[round(len(sorted_id_list)*0.7):]

In [166]:
train = result_addinfo.loc[train_id_list]
test = result_addinfo.loc[test_id_list]

In [167]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 12960 entries, 202010020706 to 202006050804
Data columns (total 17 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   着順            12960 non-null  int32         
 1   枠番            12960 non-null  int64         
 2   馬番            12960 non-null  int64         
 3   馬名            12960 non-null  object        
 4   斤量            12960 non-null  float64       
 5   騎手            12960 non-null  object        
 6   単勝            12960 non-null  float64       
 7   人気            12960 non-null  float64       
 8   性             12960 non-null  object        
 9   齢             12960 non-null  int32         
 10  体重            12960 non-null  int32         
 11  体重増減          12960 non-null  int32         
 12  course_len    12960 non-null  int32         
 13  weather       12960 non-null  object        
 14  race_type     12960 non-null  object        
 15  ground_state  12960 non

In [176]:
#関数化
def split_data(df, test_size):
    sorted_id_list = df.sort_values('date').index.unique()
    train_id_list = sorted_id_list[:round(len(sorted_id_list)*(1-test_size))]
    test_id_list = sorted_id_list[round(len(sorted_id_list)*(1-test_size)):]
    train =df.loc[train_id_list]
    test = df.loc[test_id_list]
    train.drop(['date'],axis=1,inplace=True)
    test.drop(['date'],axis=1,inplace=True)
    return train, test

In [177]:
def get_dummy(df):
    clip_rank = lambda x: x if x<4 else 4
    result = df.copy()
    result["rank"] = result["着順"].map(clip_rank)
    result.drop(["着順","馬名"],axis=1,inplace=True)
    result_d = pd.get_dummies(result)
    return result_d

In [178]:
result_d = get_dummy(result_addinfo)

In [179]:
train, test = split_data(result_d, 0.3)

In [180]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 30279 entries, 202006010101 to 202004030703
Columns: 200 entries, 枠番 to ground_state_重
dtypes: float64(3), int32(4), int64(3), uint8(190)
memory usage: 7.6+ MB


In [182]:
# from sklearn.linear_model import LogisticRegression
from imblearn.under_sampling import RandomUnderSampler

rank_1 = train["rank"].value_counts()[1]
rank_2 = train["rank"].value_counts()[2]
rank_3 = train["rank"].value_counts()[3]

rus = RandomUnderSampler(sampling_strategy={1:rank_1, 2:rank_2, 3:rank_3, 4:rank_1}, random_state=71)

X_train = train.drop(["rank"],axis=1)
y_train = train["rank"]
X_test = test.drop(["rank"],axis=1)
y_test = test["rank"]


X_train_rus, y_train_rus = rus.fit_resample(X_train, y_train)

In [186]:
from sklearn import *
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(random_state=0)
clf.fit(X_train_rus, y_train_rus)



RandomForestClassifier(random_state=0)

In [188]:
print(clf.score(X_train, y_train), clf.score(X_test, y_test))

0.7185177846031904 0.5388117283950618


In [ ]:
#訓練用データでの正解率は72%だがテストデータの正解率は53%→過学習
# これを是正するにはClassifierにパラメータを設定する必要がある